In [1]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from IPython.display import Image

No module named 'ms_deisotope._c.averagine' averagine
No module named 'ms_deisotope._c.scoring'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'


OzESI 

FA Neutral Loss m/z

In [17]:
#Create a pandas dataframe of Fatty acyl substituents starting with FA 12:0 and ending with FA 30:0
#create columns Loss_FA:1 to Loss_FA:10

df_NL_FA = pd.DataFrame(columns=['FA','Loss_FA:0','Loss_FA:1','Loss_FA:2','Loss_FA:3','Loss_FA:4','Loss_FA:5','Loss_FA:6','Loss_FA:7','Loss_FA:8','Loss_FA:9','Loss_FA:10'])
#Set FA 12:0 = 217
# increase FA by 14 for each FA
#Subtract 2 from the FA for each loss
for i in range(12,25,2):
    df_NL_FA.loc[i,'FA'] = 'FA '+str(i)
    df_NL_FA.loc[i,'Loss_FA:0'] = 217 + (i-12)*14
    for j in range(0,12,1):
        df_NL_FA.loc[i,'Loss_FA:'+str(j)] = df_NL_FA.loc[i,'Loss_FA:0'] - (j)*2
        #print(df_FA.loc[i,'Loss_FA:'+str(j)])
df_NL_FA.head(25)


,FA,Loss_FA:0,Loss_FA:1,Loss_FA:2,Loss_FA:3,Loss_FA:4,Loss_FA:5,Loss_FA:6,Loss_FA:7,Loss_FA:8,Loss_FA:9,Loss_FA:10,Loss_FA:11
12,FA 12,217,215,213,211,209,207,205,203,201,199,197,195.0
14,FA 14,245,243,241,239,237,235,233,231,229,227,225,223.0
16,FA 16,273,271,269,267,265,263,261,259,257,255,253,251.0
18,FA 18,301,299,297,295,293,291,289,287,285,283,281,279.0
20,FA 20,329,327,325,323,321,319,317,315,313,311,309,307.0
22,FA 22,357,355,353,351,349,347,345,343,341,339,337,335.0
24,FA 24,385,383,381,379,377,375,373,371,369,367,365,363.0


In [14]:
#create a pandas dataframe for TAGs starting with 36:0 to 37:10 all the way to 55:0 to 55:10
#df_TAG_precursor36 = pd.DataFrame(columns=['TAG','Loss_FA:0','Loss_FA:1','Loss_FA:2','Loss_FA:3','Loss_FA:4','Loss_FA:5','Loss_FA:6','Loss_FA:7','Loss_FA:8','Loss_FA:9','Loss_FA:10'])
df_TAG_precursor = pd.DataFrame(columns=['TAG'])
df_TAG_precursor.loc[i,'TAG 37:0'] = 670
for i in range(37,40,1):
    #iterate through all possible
    for j in range(1,11):
        df_TAG_precursor.loc[i,'TAG'] = 'TAG '+str(i) + ':' + str(j)
        df_TAG_precursor.loc[i,'TAG {}:'.format(i)+str(j)] = df_TAG_precursor.loc[i,'TAG 37:0'] - (j)*2

df_TAG_precursor.head(25)
        

,TAG,TAG 37:0,TAG 37:1,TAG 37:2,TAG 37:3,TAG 37:4,TAG 37:5,TAG 37:6,TAG 37:7,TAG 37:8,...,TAG 39:1,TAG 39:2,TAG 39:3,TAG 39:4,TAG 39:5,TAG 39:6,TAG 39:7,TAG 39:8,TAG 39:9,TAG 39:10
39,TAG 39:10,670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,668.0,666.0,664.0,662.0,660.0,658.0,656.0,654.0,652.0,650.0
37,TAG 37:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,TAG 38:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Search all TAG 37:1 - TAG 56:1 for FA 18:1 - 18:10

In [3]:
#Create a panads dataframe for TAGs starting with TAG 37:0 and ending with TAG 55:0
df_TAG1_FA18 = pd.DataFrame(columns=['TAG','Parent_Ion','Loss_18:1', 'Loss_18:2', 'Loss_18:3', 'Loss_18:4', 'Loss_18:5', 'Loss_18:6', 'Loss_18:7', 'Loss_18:8', 'Loss_18:9', 'Loss_18:10'])
#Set TAG 37:0 = 670
# increase TAG by 14 for each TAG
#Subtract the FA 18 column loss_FA in df_NL_FA from each column loss_FA in df_TAG
for i in range(37,57,1):
    df_TAG1_FA18.loc[i,'TAG'] = 'TAG '+str(i)+':1'
    df_TAG1_FA18.loc[i,'Parent_Ion'] = 670 + (i-37)*14
    for j in range(1,11,1):
        df_TAG1_FA18.loc[i,'Loss_18:'+str(j)] = df_TAG1_FA18.loc[i,'Parent_Ion'] - df_NL_FA.loc[18,'Loss_FA:'+str((j-1))]
        #print(df_TAG.loc[i,'Loss_FA:'+str(j)])
df_TAG1_FA18.head(25)



,TAG,Parent_Ion,Loss_18:1,Loss_18:2,Loss_18:3,Loss_18:4,Loss_18:5,Loss_18:6,Loss_18:7,Loss_18:8,Loss_18:9,Loss_18:10
37,TAG 37:1,670,369,371,373,375,377,379,381,383,385,387
38,TAG 38:1,684,383,385,387,389,391,393,395,397,399,401
39,TAG 39:1,698,397,399,401,403,405,407,409,411,413,415
40,TAG 40:1,712,411,413,415,417,419,421,423,425,427,429
41,TAG 41:1,726,425,427,429,431,433,435,437,439,441,443
42,TAG 42:1,740,439,441,443,445,447,449,451,453,455,457
43,TAG 43:1,754,453,455,457,459,461,463,465,467,469,471
44,TAG 44:1,768,467,469,471,473,475,477,479,481,483,485
45,TAG 45:1,782,481,483,485,487,489,491,493,495,497,499
46,TAG 46:1,796,495,497,499,501,503,505,507,509,511,513


Search TAG40:0 - 40:n for FA 18:1 - 18:n

In [4]:
#Create a panads dataframe for TAGs starting with TAG 37:0 and ending with TAG 55:0
df_TAG40_FA18 = pd.DataFrame(columns=['TAG','Parent_Ion','Loss_18:1', 'Loss_18:2', 'Loss_18:3', 'Loss_18:4', 'Loss_18:5', 'Loss_18:6', 'Loss_18:7', 'Loss_18:8', 'Loss_18:9', 'Loss_18:10'])
#Set TAG 37:0 = 670
# increase TAG by 14 for each TAG
#Subtract the FA 18 column loss_FA in df_NL_FA from each column loss_FA in df_TAG
for i in range(40,41,1):
    for z in range(1,11,1):
        df_TAG40_FA18.loc[z,'TAG'] = 'TAG '+str(i)+':'+str(z)
        df_TAG40_FA18.loc[z,'Parent_Ion'] = 670 + (i-37)*14 - (z*2)
        for j in range(1,11,1):
            df_TAG40_FA18.loc[z,'Loss_18:'+str(j)] = df_TAG40_FA18.loc[z,'Parent_Ion'] - df_NL_FA.loc[18,'Loss_FA:'+str((j))]
            #print(df_TAG.loc[i,'Loss_FA:'+str(j)])
     #   for j in range(1,11,1):
      #      df_TAG40_FA18.loc[i,'Loss_18:'+str(j)] = df_TAG40_FA18.loc[i,'Parent_Ion'] - df_NL_FA.loc[18,'Loss_FA:'+str((j-1))]
        #print(df_TAG.loc[i,'Loss_FA:'+str(j)])
df_TAG40_FA18.head(25)


,TAG,Parent_Ion,Loss_18:1,Loss_18:2,Loss_18:3,Loss_18:4,Loss_18:5,Loss_18:6,Loss_18:7,Loss_18:8,Loss_18:9,Loss_18:10
1,TAG 40:1,710,411,413,415,417,419,421,423,425,427,429
2,TAG 40:2,708,409,411,413,415,417,419,421,423,425,427
3,TAG 40:3,706,407,409,411,413,415,417,419,421,423,425
4,TAG 40:4,704,405,407,409,411,413,415,417,419,421,423
5,TAG 40:5,702,403,405,407,409,411,413,415,417,419,421
6,TAG 40:6,700,401,403,405,407,409,411,413,415,417,419
7,TAG 40:7,698,399,401,403,405,407,409,411,413,415,417
8,TAG 40:8,696,397,399,401,403,405,407,409,411,413,415
9,TAG 40:9,694,395,397,399,401,403,405,407,409,411,413
10,TAG 40:10,692,393,395,397,399,401,403,405,407,409,411


In [15]:
mrm_list_new = pd.read_excel('SUPPLE_2.XLS')


#loop through all sheets in SUPPLE_2.XLS and make a df of Compound Name, Parent Ion, and Product Ion
mrm_list_new = pd.read_excel('SUPPLE_2.XLS', sheet_name = None)
mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion']]
#Add underscore to middle of columns names
mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
#round Parent Ion and Product Ion to 1 decimal place
mrm_list_offical['Parent_Ion'] = mrm_list_offical['Parent_Ion'].round(1)
mrm_list_offical['Product_Ion'] = mrm_list_offical['Product_Ion'].round(1)
#create transition column by combining Parent Ion and Product Ion with arrow between numbers
mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
#change column compound name to lipid
mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
print(mrm_list_offical.head(25))


                                Lipid  Parent_Ion  Product_Ion      Transition
0                            LPC(2:0)       300.1        184.1  300.1 -> 184.1
1                            LPC(3:1)       312.1        184.1  312.1 -> 184.1
2                  LPC(3:0),PC(O-3:0)       314.1        184.1  314.1 -> 184.1
3        LPC(4:0),PC(O-4:0),PC(O-5:0)       328.2        184.1  328.2 -> 184.1
4                    PC(4:0),LPC(5:0)       342.2        184.1  342.2 -> 184.1
5                            LPC(6:0)       356.2        184.1  356.2 -> 184.1
6                    PC(6:0),LPC(7:0)       370.2        184.1  370.2 -> 184.1
7                            LPC(8:0)       384.2        184.1  384.2 -> 184.1
8                         LPC(O-10:1)       396.3        184.1  396.3 -> 184.1
9                    PC(8:0),LPC(9:0)       398.2        184.1  398.2 -> 184.1
10               LPC(10:0),PC(O-10:0)       412.2        184.1  412.2 -> 184.1
11                 PC(10:0),LPC(11:0)       426.3   

/tmp/ipykernel_1077/3657948472.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Parent_Ion'] = mrm_list_offical['Parent_Ion'].round(1)
/tmp/ipykernel_1077/3657948472.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Product_Ion'] = mrm_list_offical['Product_Ion'].round(1)
/tmp/ipykernel_1077/3657948472.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [27]:
search_list = pd.read_excel('search_list.xlsx')
found_list = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Transition'])
for i in range(len(mrm_list_offical)):
    for j in range(len(search_list)):
        if mrm_list_offical.loc[i,'Parent_Ion'] == search_list.loc[j,'precursor']:
           # print('yes')
            found_list = found_list.append(mrm_list_offical.loc[i,:])
            
found_list.head(25)


/tmp/ipykernel_1077/1276256897.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_list_offical.loc[i,:])
/tmp/ipykernel_1077/1276256897.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_list_offical.loc[i,:])
/tmp/ipykernel_1077/1276256897.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_list_offical.loc[i,:])
/tmp/ipykernel_1077/1276256897.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_list_offical.loc[i,:])
/tmp/ipykernel_1077/1276256897.py:7: FutureWarning: The frame.append method is d

,Lipid,Parent_Ion,Product_Ion,Transition
165,PC(44:0),902.8,184.1,902.8 -> 184.1
166,PC(46:2),926.8,184.1,926.8 -> 184.1
167,PC(46:1),928.8,184.1,928.8 -> 184.1
168,PC(46:0),930.8,184.1,930.8 -> 184.1
626,"[TG(49:8),TG(48:1)]_FA14:0",822.8,577.6,822.8 -> 577.6
637,"[TG(51:8),TG(50:1)]_FA14:0",850.8,605.6,850.8 -> 605.6
638,"[TG(51:9),TG(50:2)]_FA14:0",848.8,603.6,848.8 -> 603.6
640,[TG(52:4)]_FA14:0,872.8,627.6,872.8 -> 627.6
647,"[TG(53:8),TG(52:1)]_FA14:0",878.8,633.6,878.8 -> 633.6
648,"[TG(53:9),TG(52:2)]_FA14:0",876.8,631.6,876.8 -> 631.6
